## Import libraries

In [1]:
import sys
sys.path.insert(0, './haqs_api')

import haqs_api

%load_ext autoreload
%autoreload 2

{'init': 'epsg:4326', 'no_defs': True}

## Data addresses

In [2]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap
from fiona.crs import from_epsg
import matplotlib as mpl
import matplotlib.pyplot as plt
import bokeh
from bokeh import plotting
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.tile_providers import CARTODBPOSITRON
import psycopg2

NoneType = type(None)

%matplotlib inline

1. **Stations request** http://api.gios.gov.pl/pjp-api/rest/station/findAll
2. **Sensors request** http://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId}
3. **Data request** http://api.gios.gov.pl/pjp-api/rest/data/getData/{sensorId}
4. **AQ index request** http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/{stationId}

# Geopandas part

## Get Stations from GIOŚ API

In [4]:
stations = haqs_api.stations
print("Number of available stations: ", len(stations))

Number of available stations:  161


In [33]:
stations_df = haqs_api.stations_df
stations_df.head()

,station_id,geometry
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## Plot Stations on map

In [34]:
haqs_api.create_stations_map()

## Get list of Sensors for each Station

In [32]:
sensors_df = haqs_api.sensors_df
sensors_df.head()

,station_id,sensor_id,parameter
0,114,642,NO2
1,114,644,O3
2,117,660,CO
3,117,14395,PM10
4,117,658,C6H6


## Get Sensors readings from GIOŚ API

In [35]:
sensors_df = haqs_api.get_latest_sensors_readings()
sensors_df.head()

,station_id,sensor_id,parameter,data
0,114,642,NO2,23.41300
1,114,644,O3,85.54440
2,117,660,CO,428.58900
3,117,14395,PM10,32.20990
4,117,658,C6H6,0.00025


In [36]:
available_parameters = haqs_api.available_parameters
available_parameters

['NO2', 'O3', 'CO', 'PM10', 'C6H6', 'PM2.5', 'SO2']

In [47]:
def get_param_df(parameter=''):
    
    import geopandas as gpd
    import pandas as pd
    
    param_df = sensors_df[sensors_df["parameter"]=="{}".format(parameter)]
    param_df = gpd.GeoDataFrame(pd.merge(param_df, stations_df, on='station_id'))
    param_df.dropna(inplace=True)
    param_df.crs = haqs_api.coord_system
    
    return param_df

In [48]:
def show_readings(parameter=''):
    
    import geopandas as gpd
    import pandas as pd
    import numpy as np
    import matplotlib as mpl
    from bokeh import plotting
    from bokeh.models import GeoJSONDataSource, HoverTool
    from bokeh.plotting import figure, show, ColumnDataSource
    from bokeh.tile_providers import CARTODBPOSITRON
    
    param_df = get_param_df("{}".format(parameter))
    param_df = param_df.to_crs(epsg=3395)  # conversion to World Mercator needed
    
    plotting.output_notebook()

    # reduce differences between readings
    size=param_df["data"]
    # get data resolution
    radii = np.array(param_df['data'].apply(int))
    #create colors list
    colors = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*mpl.cm.RdYlGn(1-mpl.colors.Normalize()(radii))]

    p = plotting.figure(toolbar_location="left", 
                        plot_width=900, 
                        plot_height=700, 
                        x_axis_type="mercator", 
                        y_axis_type="mercator")

    p.circle(param_df['geometry'].x, 
             param_df['geometry'].y, 
             size=size, 
             fill_color=colors, 
             fill_alpha=0.8, 
             line_color=None)

    p.add_tile(CARTODBPOSITRON)

    plotting.show(p)

## Visualize readings

In [49]:
show_readings("O3")

Loading BokehJS ...

# Connect with database

In [23]:
try:
    conn = psycopg2.connect("dbname='haqs' user='postgres' host='localhost' password='postgres'")
except:
    print ("I am unable to connect to the database")
    conn.close()

In [22]:
conn.close()

## Create PostGIS Extension

In [29]:
sql = "CREATE EXTENSION postgis;"

cur = conn.cursor()
try:
    cur.execute(sql)
except psycopg2.ProgrammingError as e:
    print (e)

BŁĄD:  rozszerzenie "postgis" już istnieje



## <font color="blue">Show Tables</font>

In [ ]:
sql = "SELECT * FROM pg_catalog.pg_tables;"

cur = conn.cursor()
cur.execute(sql)
cur.fetchall()

## <font color="blue">Station Table</font>

### Create Stations Table

In [24]:
'''sql = """
    CREATE TABLE public.stations 
    ( 
        station_id INTEGER PRIMARY KEY
    );

    SELECT AddGeometryColumn('stations', 'geom', '4326', 'POINT', 2);
"""'''

sql = """
    CREATE TABLE public.stations 
    ( 
        station_id INTEGER PRIMARY KEY,
        geom geometry(Point, 4326)
    );
"""

cur = conn.cursor()
cur.execute(sql)

### Insert Stations into DataBase

1. Using text 
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_GeomFromText('POINT(-71.060316 48.432044)', EPSG));
2. Using lognitude and latitude
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_SetSRID(ST_MakePoint(lon, lat), EPSG));

In [25]:
stations = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/findAll").json()

In [26]:
stations[0]

{'id': 114,
 'stationName': 'Wrocław - Bartnicza',
 'gegrLat': '51.115933',
 'gegrLon': '17.141125',
 'city': {'id': 1064,
  'name': 'Wrocław',
  'commune': {'communeName': 'Wrocław',
   'districtName': 'Wrocław',
   'provinceName': 'DOLNOŚLĄSKIE'}},
 'addressStreet': 'ul. Bartnicza'}

In [27]:
def db_insert_station(conn, station_id, lon, lat):
    """
    Function inserts stations with its coordinates to Database
    """
    sql = "INSERT INTO public.stations (station_id, geom) VALUES (%s, ST_SetSRID(ST_MakePoint(%s, %s), 4326));"
    with conn.cursor() as cur:
        cur.execute(sql, (station_id, lon, lat))
    conn.commit()

In [28]:
# iterate over stations
for station in stations:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    try:
        db_insert_station(conn=conn, station_id=station_id, lon=station_lon, lat=station_lat)
    except:
        print ("Cannot execute insertion for Station ID: {}".format(station_id))

### Show Insertions

In [29]:
sql = "SELECT * FROM public.stations"

cur = conn.cursor()
cur.execute(sql)
cur.fetchall()

[(114, '0101000020E6100000E3A59BC4202431407AC37DE4D68E4940'),
 (117, '0101000020E6100000736891ED7C073140346953758F904940'),
 (129, '0101000020E6100000DA0418963F0331407E8CB96B098B4940'),
 (52, '0101000020E6100000BB809719362E30408A7780272D9A4940'),
 (109, '0101000020E6100000C07B478D094530404F3E3DB665624940'),
 (11, '0101000020E6100000C746205ED79F2E4074B515FBCB744940'),
 (14, '0101000020E6100000F4346090F4E12D400EA0DFF76F7C4940'),
 (16, '0101000020E61000003D2CD49AE6A53040F6798CF2CC5D4940'),
 (38, '0101000020E61000000FB4024356A73040C005D9B27C374940'),
 (67, '0101000020E6100000C9570229B18330400C3F389F3A4A4940'),
 (70, '0101000020E6100000DE3EABCC944A3140E9B81AD995784940'),
 (74, '0101000020E6100000A71FD4450ADD2E403F1D8F19A8A84940'),
 (84, '0101000020E610000070438CD7BC7A2F402C11A8FE415E4940'),
 (132, '0101000020E6100000FAB7CB7EDDD1304036CD3B4ED14B4940'),
 (134, '0101000020E6100000234A7B832F042E406E4E250340934940'),
 (9153, '0101000020E6100000A81C93C5FD872F40280B5F5FEB744940'),
 (10007, '010100

### Create Stations DataFrame from Stations Table

In [53]:
sql = "SELECT * FROM public.stations"

stations_df = gpd.read_postgis(sql, conn, geom_col='geom')

In [54]:
stations_df.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## <font color="blue">Sensors Table</font>

### Create Sensors Table

In [32]:
sql = """
    CREATE TABLE public.sensors 
    (
        sensor_id INTEGER PRIMARY KEY,
        sensor_parameter VARCHAR(10) NOT NULL,
        station_id INTEGER REFERENCES stations (station_id)
    );
"""
    
cur = conn.cursor()
cur.execute(sql)

### Insert Sensors into DataBase

> <i>**INSERT INTO**</i> sensors(sensor_id, sensor_parameter, station_id) **VALUES** (2, 'PM10', 114));

In [33]:
def db_insert_station_sensors(conn, sensor_id, sensor_parameter, station_id):
    """
    Function inserts sensors for each station to Database
    """
    sql = """INSERT INTO public.sensors (sensor_id, sensor_parameter, station_id)
                VALUES (%s, %s, %s);"""
    
    with conn.cursor() as cur:
        cur.execute(sql, (sensor_id, sensor_parameter, station_id))
    conn.commit()

In [34]:
# iterate over stations
for station in stations:
    
    station_id = station["id"]
    sensors = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    # iterate over station sensors
    for sensor in sensors:
        
        sensor_id = sensor["id"]
        sensor_parameter = sensor["param"]["paramCode"]
        
        try:
            db_insert_station_sensors(conn, sensor_id, sensor_parameter, station_id)
        except Exception as e:
            print(e)
            print ("Cannot execute insertion for Sensor ID: {} (Station ID: {})".format(sensor_id, station_id))

### Create Sensors DataFrame from Sensors Table

In [55]:
sql = "SELECT * FROM public.sensors"

sensors_df = pd.read_sql_query(sql,con=conn)
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


## <font color="blue">Readings Table</font>

### Create Readings Table  

In [36]:
sql = """
    CREATE TABLE public.readings 
    (
        sensor_id INTEGER REFERENCES sensors (sensor_id),
        date VARCHAR(19) NOT NULL,
        reading FLOAT(4)
    );
"""
    
cur = conn.cursor()
cur.execute(sql)

### Create list of available Sensors

In [38]:
sql = "SELECT sensor_id FROM sensors;"

cur = conn.cursor()
cur.execute(sql)
sensors_ids = [values[0] for values in cur.fetchall()]
sensors_ids[:2]

[642, 644]

### Insert Readings Into DataBase

> <i><b>INSERT INTO**</b></i> readings(sensor_id, date, reading) <i><b>VALUES</b></i> (621, '2018-09-05 00:00:00', 32.2259));

In [39]:
def db_insert_sensor_readings(conn, sensor_id, date, reading):
    """
    Function inserts multiple readings for sensor
    """
    
    sql = """
        INSERT INTO readings (sensor_id, date, reading)
        SELECT %s, %s, %s
        WHERE
            NOT EXISTS 
            (
                SELECT * FROM readings WHERE date = %s AND sensor_id = %s
            )
    """
    
    with conn.cursor() as cur:
        cur.execute(sql, (sensor_id, date, reading, date, sensor_id))
    conn.commit()

#### Insert all available non NULL readings into Readings Table

In [40]:
for sensor_id in sensors_ids:

    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    data = data_json['values']
    
    # check all available data
    for row in data:

        date = row['date']
        reading = row['value']

        # if there is no value wait until next attempt
        if isinstance(reading, NoneType):
            continue

        try:
            db_insert_sensor_readings(conn, sensor_id, date, reading)
        except Exception as e:
            print(e)

### Create Readings DataFrame from Readings Table

In [56]:
sql = "SELECT * FROM public.readings"

readings_df = pd.read_sql_query(sql,con=conn)
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [42]:
readings_df.describe()

,sensor_id,reading
count,41096.000000,41096.000000
mean,6907.856361,64.441994
std,5898.822276,134.640051
min,50.000000,0.000000
25%,2649.000000,4.670325
50%,4777.000000,18.308200
75%,14395.000000,42.200000
max,19816.000000,2928.870000


In [43]:
readings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41096 entries, 0 to 41095
Data columns (total 3 columns):
sensor_id    41096 non-null int64
date         41096 non-null object
reading      41096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 963.3+ KB


## Create and save GeoDataFrame to Shapefile

In [57]:
stations_df.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


In [58]:
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


In [59]:
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [60]:
sensors_df = sensors_df.merge(stations_df, on='station_id')
sensors_df.head()

,sensor_id,sensor_parameter,station_id,geom
0,642,NO2,114,POINT (17.141125 51.115933)
1,644,O3,114,POINT (17.141125 51.115933)
2,660,CO,117,POINT (17.02925 51.129378)
3,14395,PM10,117,POINT (17.02925 51.129378)
4,658,C6H6,117,POINT (17.02925 51.129378)


In [61]:
readings_df = gpd.GeoDataFrame(readings_df.merge(sensors_df, on='sensor_id'))
readings_df.crs = from_epsg(4326)
readings_df.rename(index=str, columns={"geom": "geometry"}, inplace=True)
readings_df.head()

,sensor_id,date,reading,sensor_parameter,station_id,geometry
0,642,2018-09-07 18:00:00,12.93730,NO2,114,POINT (17.141125 51.115933)
1,642,2018-09-07 17:00:00,12.83960,NO2,114,POINT (17.141125 51.115933)
2,642,2018-09-07 16:00:00,11.44120,NO2,114,POINT (17.141125 51.115933)
3,642,2018-09-07 15:00:00,9.43838,NO2,114,POINT (17.141125 51.115933)
4,642,2018-09-07 14:00:00,10.22020,NO2,114,POINT (17.141125 51.115933)


In [62]:
readings_df.to_file('shp/readings.shp')